# Foodvocate: Asynchronous Independent Cascade Model (AsIC)
Author: [Meng-Chieh Liu](https://github.com/MengChiehLiu)  
Date: 2023/6/2

In [68]:
from Instagram.data.datasets import datasets, bloggers, frequency_count
import numpy as np
from scipy.optimize import minimize
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [69]:
# config
n = 578 # number of unique nodes
d = 91 # number of datasets
max_timeDecay = 30 # maximum time decay days
beta_theta = np.zeros(n*4) # initialize value
connected_node = 24 # average node in each dataset

In [70]:
# weighted g: the possibility of not in an document
frequency = np.array(list(frequency_count.values()))
show = frequency/d
no_show = 1 - show

In [71]:
bloggers_index = {v:k for k,v in bloggers.items()}

In [72]:
new_datasets = []

for dataset in datasets:
    new_dataset = []
    first_time_stamp = dataset[0][1]
    for user_name, time_stamp in dataset:
        new_dataset.append((bloggers_index[user_name], (time_stamp-first_time_stamp)/86400))
    new_datasets.append(new_dataset)

In [73]:
A = []
T = []
W = []

for new_dataset in new_datasets:
    Activated = []
    TimeDacays = []
    w = []

    for i in range(1, len(new_dataset)): # node be triggered (after)
        node2, time2 = new_dataset[i]
        activated = np.array([])
        timeDacays = np.array([])

        for j in range(i): # node trigger (before)
            node1, time1 = new_dataset[j]
            if time2-time1 <= max_timeDecay:
                x = np.zeros(n*2)
                x[node1] = 1
                x[n+node2] = 1
                activated = np.concatenate([activated, x])
                timeDacays = np.append(timeDacays, time2-time1)

        if timeDacays.size > 0:
            w.append(node2)
            Activated.append(activated.reshape(-1, n*2))
            TimeDacays.append(timeDacays)
    
    A.append(Activated)
    T.append(TimeDacays)
    W.append(w)

In [74]:
diagonal = np.eye(n)
g_matrixs = []
for i in range(n):
    g_matrix = np.zeros((n,n))
    g_matrix[:, i] = 1
    g_matrix = np.concatenate((g_matrix, diagonal), axis=1)
    g_matrixs.append(g_matrix)

In [75]:
### beta's grad ###
# beta here is np.dot(X, beta)
# p = sigmoid(beta)
# r = np.exp(theta)
# prob = p*r*np.exp(-r*time_dacays)
# neg_prob = p*np.exp(-r*time_dacays) + (1-p)

# d(prob) / d(p) = r * np.exp(-r * time_dacays)  
# d(neg_prob) / d(p) = np.exp(-r * time_dacays) - 1
# d(p) / d(beta) = p * (1-p)
# d(prob) / d(beta) = r * np.exp(-r * time_dacays) * p * (1-p)
#                   = prob * (1-p)
# d(neg_prob) / d(beta) = (np.exp(-r * time_dacays) - 1) * p * (1-p)


### theta's grad ###
# theta here is np.dot(X, theta)
# p = sigmoid(beta)
# r = np.exp(theta)
# prob = p*r*np.exp(-r*time_dacays)
# neg_prob = p*np.exp(-r*time_dacays) + (1-p)

# d(prob) / d(r) = (d(p*r)/d(r) * np.exp(-r*time_dacays)) + (d(np.exp(-r*time_dacays))/d(r)* (p*r))
#                = p*np.exp(-r*time_dacays) + -time_dacays*np.exp(-r*time_dacays)*(p*r)
#                = (p - p*r*time_dacays) * np.exp(-r*time_dacays)
#                = p * (1 - r * time_dacays) * np.exp(-r * time_dacays)
# d(neg_prob) / d(r) = d(neg_prob)/d(np.exp(-r*time_dacays)) * d(np.exp(-r*time_dacays))/d(r)
#                    = p * (-time_dacays * np.exp(-r * time_dacays))
#                    = -p * time_dacays * np.exp(-r * time_dacays)
# d(r) / d(theta) = r
# d(prob) / d(theta) = p * (1 - r * time_dacays) * np.exp(-r * time_dacays) * r
#                    = prob * (1 - r * time_dacays)
# d(neg_prob) / d(theta) = -p * time_dacays * np.exp(-r * time_dacays) * r

In [76]:
def sigmoid(x):
    p = x.copy()
    p[x >= 0] = 1.0 / (1 + np.exp(-x[x >= 0]))
    p[x < 0] = np.exp(x[x < 0]) / (1 + np.exp(x[x < 0])) # alternative method
    return p

def cal_prob(p, r, time_dacays, idx):
    prob = p*r*np.exp(-r*time_dacays) * show[idx]
    beta_grad = prob * (1-p) * show[idx]
    theta_grad = prob * (1 - r * time_dacays) * show[idx]
    return prob, beta_grad, theta_grad

def cal_neg_prob(p, r, time_dacays, idx):
    neg_prob = (p*np.exp(-r*time_dacays)+ (1-p)) * show[idx]
    neg_beta_grad = (np.exp(-r * time_dacays) - 1) * p * (1-p) * show[idx]
    neg_theta_grad = -p * time_dacays * np.exp(-r * time_dacays) * r * show[idx]
    return neg_prob, neg_beta_grad, neg_theta_grad

In [77]:
### h' grad
# beta here is np.dot(X, beta)
# a = np.log(neg_probs).sum(), b = np.log(const)
# const = (probs/neg_probs).sum()

# d(a) / d(beta) = d(a)/d(neg_probs) * d(neg_probs)/d(beta)
#                = ((1/neg_probs) * neg_beta_grad).sum()
# d(b) / d(beta) = d(b) / d(const) * d(const) / d(beta)
#                = 1/const * (d(probs)/d(beta) * neg_probs - probs * d(neg_probs)/d(beta)) / neg_probs*neg_probs
#                = 1/const * (beta_grad*neg_probs - probs*neg_beta_grad)/neg_probs*neg_probs
# d(h) / d(beta) = d(a)/d(beta) + d(b)/d(beta) 
#                = ((1/neg_probs) * neg_beta_grad).sum() + 1/const * (beta_grad*neg_probs - probs*neg_beta_grad)/neg_probs*neg_probs

# thete is the same as beta

In [78]:
def cal_h_grad(x, const, probs, neg_probs, beta_grad, neg_beta_grad, theta_grad, neg_theta_grad):
    beta_grad = (1/neg_probs)*neg_beta_grad + 1/const * (beta_grad*neg_probs - probs*neg_beta_grad)/neg_probs*neg_probs
    theta_gard = (1/neg_probs)*neg_theta_grad + 1/const * (theta_grad*neg_probs - probs*neg_theta_grad)/neg_probs*neg_probs
    return np.concatenate((np.dot(beta_grad, x), np.dot(theta_gard, x)))

In [79]:
### g' grad
# beta here is np.dot(X, beta)
# p = sigmoid(beta)
# g_probs = show*(1-p) + no_show
# g = log(g_probs)*c

# d(g_probs)/d(beta) = d(g_probs)/d(p) * d(p)/d(beta)
#                    = -show * p*(1-p)
# d(g) / d(beta) = d(g)/d(g_probs) * d(g_probs)/d(beta)
#                = c/g_probs * (-show*p*(1-p))

In [80]:
# g: Given that node1 existed, the probability that node1 did not trigger node2 can be divided to:
# 1. node2 show up : show[node2] * (1-p)
# 2. node2 not show up : no_show[node2]

In [81]:
def negative_log_likelihood(beta_theta):
    beta = beta_theta[:n*2]
    theta = beta_theta[n*2:]

    log_likelihood = 0
    gradients = np.zeros_like(beta_theta)  # initialize gradients

    for Activated, TimeDacays, w in zip(A, T, W):

        for x, dacays, idx in zip(Activated, TimeDacays, w):
            p = sigmoid(np.dot(x, beta))
            r = np.exp(np.dot(x, theta))
            
            probs, beta_grad, theta_grad = cal_prob(p, r, dacays, idx) # X
            neg_probs, neg_beta_grad, neg_theta_grad = cal_neg_prob(p, r, dacays, idx) # Y
            
            # calculate h
            const = (probs/neg_probs).sum()
            h = (np.log(neg_probs).sum() + np.log(const))
            h_grad = cal_h_grad(x, const, probs, neg_probs, beta_grad, neg_beta_grad, theta_grad, neg_theta_grad)

            # update
            log_likelihood -= h
            gradients -= h_grad
    
    # calculate g
    for i in range(n):
        c = np.ones(n)
        c[i] = 0
        p = sigmoid(np.dot(g_matrixs[i], beta))
        g_probs = show*(1-p) + no_show
        g = (np.log(g_probs)*c).sum() * frequency_count[i] * (n-connected_node)/n
        g_grad = c/g_probs * (-show*p*(1-p)) * frequency_count[i] * (n-connected_node)/n
        g_grad = np.dot(g_grad, g_matrixs[i])
        log_likelihood -= g
        gradients[:n*2] -= g_grad

    print(f'\rlog_likelihood: {log_likelihood}', end='')
    return log_likelihood, gradients

In [82]:
maxmize_log_likelihood = minimize(negative_log_likelihood, x0=beta_theta, method='BFGS',
                                  jac=True, options={'gtol':1e-3, 'maxiter':100})

log_likelihood: 29530.735887694656

In [83]:
optimized_beta_theta = maxmize_log_likelihood.x

## Probability Matrix

In [85]:
optimized_beta = optimized_beta_theta[:n*2]
optimized_theta = optimized_beta_theta[n*2:]

In [86]:
probability_matrix = np.array([])

for i, g_matrix in enumerate(g_matrixs):
    
    prob_row = sigmoid(np.dot(g_matrix, optimized_beta)) * show
    probability_matrix = np.concatenate((probability_matrix, prob_row))

probability_matrix = probability_matrix.reshape(-1, n)

In [87]:
df = pd.DataFrame(probability_matrix)
for i in range(n):
    df[i][i] = 0

In [88]:
df[0].sort_values().head()

0      0.000000
3      0.001296
53     0.001514
362    0.001774
287    0.001802
Name: 0, dtype: float64

In [100]:
df[0].sort_values().tail()

107    0.004955
238    0.004962
33     0.004979
61     0.005102
24     0.005150
Name: 0, dtype: float64

In [90]:
df.to_csv('data/probability_matrix.csv')